In [3]:
import numpy as np

# Definizione delle esposizioni (in milioni di euro)
esposizione_azionario = 100
esposizione_obbligazionaria = 200
esposizione_immobiliare = 50
esposizione_valutaria = 30
esposizione_spread = 100
esposizione_concentrazione = 80
esposizione_controparti = [50, 30, 40]  # Esposizioni a diverse controparti
prob_default = [0.02, 0.03, 0.015]  # Probabilità di default delle controparti
loss_given_default = [0.6, 0.5, 0.7]  # LGD per ciascuna controparte

# Esposizioni per rischi assicurativi
esposizione_vita = 500  # Life Underwriting Risk
esposizione_non_vita = 300  # Non-Life Underwriting Risk
esposizione_salute = 200  # Health Underwriting Risk

# Shock regolatori
shock_azionario = 0.39
shock_tasso = 0.015
shock_immobiliare = 0.25
shock_valutario = 0.25
shock_spread = 0.084
soglia_concentrazione = 50
tasso_concentrazione = 0.10
shock_vita = 0.15  # 15% di shock su esposizioni Vita
shock_non_vita = 0.20  # 20% su esposizioni Non-Vita
shock_salute = 0.18  # 18% su esposizioni Health

# Calcolo SCR per rischio di mercato
scr_azionario = esposizione_azionario * shock_azionario
scr_tasso = esposizione_obbligazionaria * shock_tasso * 8
scr_immobiliare = esposizione_immobiliare * shock_immobiliare
scr_valutario = esposizione_valutaria * (1 - 1 / (1 + shock_valutario))
scr_spread = esposizione_spread * shock_spread
scr_concentrazione = max(0, (esposizione_concentrazione - soglia_concentrazione) * tasso_concentrazione)

# Calcolo SCR per rischio di controparte
scr_controparti = sum(e * p * l for e, p, l in zip(esposizione_controparti, prob_default, loss_given_default))

# Calcolo SCR per rischi assicurativi
scr_vita = esposizione_vita * shock_vita
scr_non_vita = esposizione_non_vita * shock_non_vita
scr_salute = esposizione_salute * shock_salute

# Generazione matrice di correlazione casuale
np.random.seed(42)  # Per riproducibilità
dim = 9
corr_matrix = np.random.normal(0.5, np.sqrt(3), (dim, dim))
corr_matrix = (corr_matrix + corr_matrix.T) / 2  # Rende la matrice simmetrica
np.fill_diagonal(corr_matrix, 1)  # Imposta la diagonale a 1

# Proietta i valori tra -1 e 1
corr_matrix = np.clip(corr_matrix, -1, 1)

# Assicura che la matrice sia semi-definita positiva
vals, vecs = np.linalg.eigh(corr_matrix)
vals = np.clip(vals, 0, None)  # Impedisce valori negativi
d_corr = vecs @ np.diag(vals) @ vecs.T  # Ricostruisce la matrice

# Calcolo SCR totale per i tre blocchi separati
scr_sotto_moduli_mercato = np.array([scr_tasso, scr_azionario, scr_immobiliare, scr_valutario, scr_spread, scr_concentrazione])
scr_mercato = np.sqrt(scr_sotto_moduli_mercato @ d_corr[:6, :6] @ scr_sotto_moduli_mercato.T)

scr_sotto_moduli_controparti = np.array([scr_controparti])
scr_controparti_finale = scr_controparti  # Non ha una matrice di correlazione complessa

scr_sotto_moduli_underwriting = np.array([scr_vita, scr_non_vita, scr_salute])
scr_underwriting = np.sqrt(scr_sotto_moduli_underwriting @ d_corr[6:, 6:] @ scr_sotto_moduli_underwriting.T)

# Formula finale per lo SCR aggregato
scr_totale = np.sqrt(scr_mercato**2 + scr_controparti_finale**2 + scr_underwriting**2)

# Stampa risultati
print("Matrice di correlazione casuale:")
print(d_corr)
print(f"\nSCR Totale Rischio di Mercato: {scr_mercato:.2f}M")
print(f"SCR Totale Rischio di Controparte: {scr_controparti_finale:.2f}M")
print(f"SCR Totale Rischi Assicurativi: {scr_underwriting:.2f}M")
print(f"SCR Totale Aggregato: {scr_totale:.2f}M")

Matrice di correlazione casuale:
[[ 1.3357794   0.74829418 -0.01673419  0.68801298  0.4309279  -0.14625116
   0.71834343  0.35307841 -0.05913668]
 [ 0.74829418  1.64502467 -0.72485304 -0.00851955 -0.42393063 -0.48724094
   0.44376517 -0.00819669  0.84777491]
 [-0.01673419 -0.72485304  1.50868528  0.16965202 -0.20927425  0.1677088
  -0.56239439  0.60929749 -0.73230287]
 [ 0.68801298 -0.00851955  0.16965202  1.44968513  0.72152859  0.35825217
  -0.21261012  0.95535847  0.2870202 ]
 [ 0.4309279  -0.42393063 -0.20927425  0.72152859  1.6667532  -0.41082136
   0.33166842  0.62606707 -0.6725955 ]
 [-0.14625116 -0.48724094  0.1677088   0.35825217 -0.41082136  1.62513077
   0.41816766  0.11840173  0.49743161]
 [ 0.71834343  0.44376517 -0.56239439 -0.21261012  0.33166842  0.41816766
   1.58770233 -0.22579272 -0.09961175]
 [ 0.35307841 -0.00819669  0.60929749  0.95535847  0.62606707  0.11840173
  -0.22579272  1.33207067 -0.01523241]
 [-0.05913668  0.84777491 -0.73230287  0.2870202  -0.6725955   0